<a href="https://colab.research.google.com/github/Dreilly5/readme-img/blob/main/FakeNewsDataProjectGroup2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
import nltk
from google.colab import drive
import os
warnings.filterwarnings('ignore')
%matplotlib inline

/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

In [ ]:
#THIS IS FOR THIERRY's Implementation only
drive.mount("/content/drive")

Mounted at /content/drive


#Store The files as variables

In [ ]:

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
test_labels_raw = pd.read_csv('submit.csv')


In [ ]:
#This is for Thierry's implementation only
training = '/content/drive/MyDrive/FakeNewsData/train.csv'
testing = '/content/drive/MyDrive/FakeNewsData/test.csv'
train_data = pd.read_csv(training)
test_data = pd.read_csv(testing)
print(os.listdir('/content/drive/MyDrive/FakeNewsData/'))
test_labels_raw = pd.read_csv('/content/drive/MyDrive/FakeNewsData/submit.csv')

# Drop The Empty Values

In [ ]:
train_data.info()

In [ ]:
# prompt: drop the train_data and test_data with null text data

train_data.dropna(subset=['text'], inplace=True)
test_data.dropna(subset=['text'], inplace=True)

print (train_data.info())


##Add Column for Final news with title and text

In [ ]:
print (train_data.shape)
print (test_data.shape)

train_data['news'] = train_data['title'] + ' ' + train_data['text']
test_data['news'] = test_data['title'] + ' ' + test_data['text']

print (train_data.shape)
print (test_data.shape)
train_data.head()

##Drop useless columns

In [ ]:
delete_columns = ['title', 'text', 'author']
train_data = train_data.drop(delete_columns, axis=1)
test_data = test_data.drop(delete_columns, axis=1)
train_data.head()


##Next we need to filter out the (special chars and extra space and newlines)

In [ ]:
train_data['news'] = train_data['news'].str.lower()
test_data['news'] = test_data['news'].str.lower()
train_data['news'] = train_data['news'].str.replace("[^a-zA-Z0-9\s]", "")
train_data['news'] = train_data['news'].str.replace("\n", "")
train_data['news'] = train_data['news'].str.replace('\s+','')
test_data['news'] = test_data['news'].str.replace("[^a-zA-Z0-9/s]", "")
test_data['news'] = test_data['news'].str.replace("\n", "")
test_data['news'] = test_data['news'].str.replace("\s+", "")
train_data.head()
test_data.head()

In [ ]:
# prompt: print out train data news index 4

print(train_data['news'][4])


#Remove Float Types

In [ ]:
train_data['news'] = train_data['news'].astype(str)
test_data['news'] = test_data['news'].astype(str)


#Removing Stopwords

In [ ]:
# prompt: remove stopwords

from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

train_data['news'] = train_data['news'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
test_data['news'] = test_data['news'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


# Turning Data Into Tensors

In [ ]:
print(type(train_data))
print(type(train_data['news']))
batchsize = 4

In [ ]:
#Tokenizing training data
class NewsDataset(Dataset):
    def __init__(self, padded_sequences, labels):
        self.padded_sequences = padded_sequences
        if isinstance(labels, torch.Tensor):
            self.labels = labels.unsqueeze(1) if len(labels.shape) == 1 else labels
        else:
            self.labels = torch.tensor(labels.values, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.padded_sequences)

    def __getitem__(self, idx):
        return self.padded_sequences[idx], self.labels[idx]

# Tokenizing training data
tokenizer = get_tokenizer("basic_english")
train_tokens = train_data['news'].apply(tokenizer)

# Build vocabulary
train_vocab = build_vocab_from_iterator(train_tokens, specials=["<unk>"])
train_vocab.set_default_index(train_vocab["<unk>"])

def numerical_encoding(token_list):
    return train_vocab(token_list)

train_sequences = train_tokens.apply(numerical_encoding)

# Pad sequences
train_padded_sequences = torch.nn.utils.rnn.pad_sequence(
    [torch.tensor(seq) for seq in train_sequences], batch_first=True
)

# Convert labels to tensor
train_labels = torch.tensor(train_data["label"].values, dtype=torch.float32)  # Ensure dtype matches


# Create dataset and dataloader
train_dataset = NewsDataset(train_padded_sequences, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True, num_workers=4)

In [ ]:
print(type(train_labels))

In [ ]:
#Tokenizing testing data
tokenizer = get_tokenizer("basic_english")
test_tokens = test_data['news'].apply(tokenizer)

# Build vocabulary (ensure it matches with training vocabulary or use the same vocab)
test_vocab = build_vocab_from_iterator(test_tokens, specials=["<unk>"])
test_vocab.set_default_index(test_vocab["<unk>"])

test_sequences = test_tokens.apply(numerical_encoding)

# Pad sequences
test_padded_sequences = torch.nn.utils.rnn.pad_sequence(
    [torch.tensor(seq) for seq in test_sequences], batch_first=True
)

# Convert labels to tensor
test_labeled = test_labels_raw['label'].astype(int)
labels = torch.tensor(test_labeled.values, dtype=torch.float32)  # Ensure dtype matches
test_labels = labels.unsqueeze(1)

# Prepare test dataset and dataloader
test_labels = test_labels_raw['label']
test_dataset = NewsDataset(test_padded_sequences, test_labels)
test_loader = DataLoader(test_dataset, batch_size=batchsize, shuffle=False, num_workers=4)

In [ ]:
print(test_padded_sequences[1].size())
print(train_padded_sequences[1].size())
print(type(train_padded_sequences))

In [ ]:
desired_length = train_padded_sequences[1].size(0)

# Function to pad a single sequence tensor to the desired length
def pad_sequence(seq_tensor, desired_length, padding_value):
    length = seq_tensor.size(0)
    if length < desired_length:
        padding = (0, desired_length - length)
        return torch.nn.functional.pad(seq_tensor, padding, value=padding_value)
    else:
        return seq_tensor[:desired_length]

test_padded_sequences = torch.stack([pad_sequence(seq, desired_length, test_vocab["<unk>"]) for seq in test_padded_sequences])


In [ ]:
print(test_padded_sequences.shape)
print(train_padded_sequences.shape)

# Building A Basic RNN

In [ ]:
# Function to build RNN model in general
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size, num_layers):
        super(RNNModel, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        # Define the embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Define the RNN layer
        self.rnn = nn.RNN(input_size=embedding_dim,  # Use embedding_dim as input size
                          hidden_size=hidden_size,
                          num_layers=num_layers,
                          batch_first=True,
                          nonlinearity='relu')  # Use ReLU activation function

        # Define the output layer
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Apply embedding layer
        x = self.embedding(x)  # Convert token indices to embeddings

        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Pass through RNN
        out, _ = self.rnn(x, h0)

        # Pass through output layer
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)

        return out

In [ ]:
# Declaring first preliminary model
input_size = 64
hidden_size = 64
output_size = 1
num_layers = 6

first_model = RNNModel(len(train_vocab), input_size, hidden_size, output_size, num_layers)

# Running The Initial RNN

In [ ]:
print(type(train_loader))
print(type(test_loader))
for inputs, labels in train_loader:
    print(f'Batch input shape: {inputs.shape}')
    print(f'Batch label shape: {labels.shape}')
    break

In [ ]:

criterion = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(first_model.parameters(), lr=0.001)


num_epochs = 10

first_train_acc = []
first_train_loss = []
first_val_acc = []
first_val_loss = []

for epoch in range(num_epochs):
    # Training phase
    first_model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = first_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss = train_loss / total_train
    train_accuracy = correct_train / total_train

    first_train_loss.append(train_loss)
    first_train_acc.append(train_accuracy)

    # Validation phase
    first_model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            print(inputs)
            outputs = first_model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss = val_loss / total_val
    val_accuracy = correct_val / total_val

    first_val_loss.append(val_loss)
    first_val_acc.append(val_accuracy)

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')